In [1]:

# %pip install imageio[ffmpeg]

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import os

from model1 import Model1
from endovis_dataset import EndovisDataset
from reformat_data import convert_all_data

segmentation_model_weights_path = "model1_weights.pth"
tracking_model_weights_path = "tracking_model_weights.pth"

og_data_path = r"original_data"
data_path = r"data"


print(os.getcwd())

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Obtaining dependency information for imageio-ffmpeg from https://files.pythonhosted.org/packages/a0/2d/43c8522a2038e9d0e7dbdf3a61195ecc31ca576fb1527a528c877e87d973/imageio_ffmpeg-0.6.0-py3-none-manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.5/29.5 MB 294.6 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
/storage/ice1/5/4/arothe6/CS_7643_project


In [2]:
# Converts all the data from video to npz files
# Only need to run this once when setting up your system
# It will probably take 10 minutes to run; be patient :)

# Subfolders where data is; True indicates it is segmentation data
subfolders = { 
    "segmentation_train": True,
    "segmentation_test": True,
    "tracking_train": False,
    "tracking_test": False
}
convert_all_data(og_data_path, data_path, subfolders)

original_data/segmentation_train/Training/Dataset2/Video1.avi
original_data/segmentation_train/Training/Dataset2/Segmentation1.avi
original_data/segmentation_train/Training/Dataset3/Video1.avi
original_data/segmentation_train/Training/Dataset3/Segmentation1.avi
original_data/segmentation_train/Training/Dataset1/Video1.avi
original_data/segmentation_train/Training/Dataset1/Left_Instrument_Segmentation1.avi
original_data/segmentation_train/Training/Dataset1/Right_Instrument_Segmentation1.avi
original_data/segmentation_train/Training/Dataset4/Video1.avi
original_data/segmentation_train/Training/Dataset4/Segmentation1.avi
original_data/segmentation_test/Dataset5/Video1.avi
original_data/segmentation_test/Dataset5/Left_Instrument_Segmentation1.avi
original_data/segmentation_test/Dataset5/Right_Instrument_Segmentation1.avi
original_data/segmentation_test/Dataset2/Video1.avi
original_data/segmentation_test/Dataset2/Segmentation1.avi
original_data/segmentation_test/Dataset3/Video1.avi
original

In [1]:
# Training for Segmentation

# Hyperparameters 
# TODO tune these! I just made them randomly
batch_size = 16
learning_rate = 0.0001
momentum = 0.9
epochs = 10

# Model
model = Model1(4) # Switch out with whichever model you like

folder = os.path.join(data_path, "segmentation_train")
dataset = EndovisDataset(folder)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True) # TODO can tune batch size

criterion = nn.BCELoss() # Because https://medium.com/@kitkat73275/multi-label-classification-8d8ae55e8373#:~:text=For%20multi%2Dlabel%20classification%20with,class%20labels%20for%20new%20instances.
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum) # TODO tune these hyperparameters

for epoch in range(epochs):
    print("Epoch %d:" % epoch)
    for frames, truths in dataloader:
        optimizer.zero_grad()
        outputs = model(frames)
        loss = criterion(outputs, truths)
        loss.backward()
        optimizer.step()
    print("Loss: %f" % loss.item())

torch.save(model.state_dict(), segmentation_model_weights_path)


NameError: name 'Model1' is not defined

In [ ]:
# Testing for Segmentation

model = Model1()
model.load_state_dict(torch.load(segmentation_model_weights_path))
model.eval()

test_folder = r"data\segmentation_test"
testset = EndovisDataset(test_folder)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=True) 

# Test the model
with torch.no_grad():
    for frames, truths in testloader:
        outputs = torch.sigmoid(model(frames)) # Apply sigmoid for probabilities
        # TODO Evaluate performance, like IOU or DICE

In [ ]:
# Training for Tracking

# TODO Cloud
# Make a new model in a new file
# Train it here

In [ ]:
# Testing for Tracking

# TODO Cloud
# Test the new model